In [21]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from scipy.sparse import csr_matrix
# import faiss
import random

warnings.filterwarnings(action='ignore')
# books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
# books = books.replace({np.nan: 'none'})
books = pd.read_csv('/content/drive/MyDrive/books_mod2', encoding='utf-8')
books = books.replace({np.nan: 'none'})

# 데이터프레임이 content, thumbnail column을 포함한다면
# books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]
# books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]

#books_df[['description']=='none']=None
books_df_nax=books.dropna(axis='rows')


tfidf = TfidfVectorizer(stop_words='english')
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
# title_vect1 = cnt_vect.fit_transform(books_df_nax['title'])
# title_vect2 = tfidf.fit_transform(books_df_nax['title'])
#des_vect = cnt_vect.fit_transform(books['description'])
#books['book_info']=books['title']+books['description']
#b_category=books['kdc_class_no']

# 차원 축소 part 를 줄여야 함 -> 파일을 새로 저장해야할듯!
#X = title_vect2
#svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
#X_new=svd.fit_transform(X)

#D = X_new.shape[1]
#K = 500  # The number of clusters
#X = X_new.astype(np.float32)

# Setup
#kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True,gpu=True)
# For GPU(s), run the following line. This will use all GPUs
# kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True, gpu=True)

# Run clustering
#kmeans.train(X)

# Error for each iteration
# print(kmeans.obj)  # array with 20 elements

# Centroids after clustering
# print(kmeans.centroids.shape)  # (10, 500)

# The assignment for each vector.
#dists, ids = kmeans.index.search(X, 1)  # Need to run NN search again


#books_df_nax['tit_cluster']=ids

#books_df_nax

def find_sim_books(U):
    def userbooks(A):
        x=books_df_nax['isbn'].tolist()
        l=len(A)
        BD=[]
        for i in range(l):
            if A[i] in x:
              BD.append(A[i])
        l2=len(BD)
        n=random.randrange(0,l2)
        return BD[n]

    # 사용자의 isbn 데이터를 담은 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
    # B = 사용자 데이터에서 랜덤 추출한 책의 isbn 
    B=userbooks(U) 


    def sim_idx_with_A(df,isbn):
        n=int(df[df['isbn']==isbn]['tit_cluster'])
        same_clu_books_df=df[df['tit_cluster']==n]
        sen = same_clu_books_df['content']

        # 1번 후보 : sentence transformer 유사도 구하기
        #from sentence_transformers import SentenceTransformer
        #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        #sen_embeddings = model.encode(sen)
    
        # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
        clu_tfidf = tfidf.fit_transform(sen)
        clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)

        # idx_list=same_clu_books_df.index[same_clu_books_df['title']==books].tolist()
        idx=np.where(same_clu_books_df['isbn']==B)[0][0]
        books_sim_vect=clu_des_sim[idx:idx+1]
        books_des_sim_idx = books_sim_vect.argsort()[::-1]

        return books_des_sim_idx

    # 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
    sim_books_idx=sim_idx_with_A(books_df_nax,B)

    #books=userbooks(A)
    #n=int(df[df['title']==books]['tit_cluster'])
    #same_clu_books_df=df[df['tit_cluster']==n]
    #sen = same_clu_books_df['description']
    top_n=20
    top_sim_idx=sim_books_idx[0][:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)

    sim_books=books_df_nax.iloc[top_sim_idx]
    # 데이터프레임이 content, thumbnail column을 포함한다면
    outputs=sim_books[['isbn','title','author','content','thumbnail','publisher','kdc','price']]
    # outputs=sim_books[['title','author','img_url','publisher','kdc_class_no','price']]
    outputs_dic=outputs.to_dict('records')

    return outputs_dic

In [5]:
x=books_df_nax['isbn'].tolist()
x

[9791168150102,
 9791190616454,
 9791165041465,
 9791161151458,
 8809505727007,
 9791191534238,
 9791136740441,
 9791136744791,
 9791136744807,
 9780756655068,
 9781605372273,
 9781510101456,
 9781420958096,
 9781946748065,
 9781683041979,
 9780914357889,
 9783775716130,
 9781558590106,
 9780295987705,
 9781555951610,
 9781877372001,
 9780892367573,
 9780811805063,
 9783775709163,
 9780942642124,
 9781858941974,
 9781893164154,
 9780894940606,
 9780711224292,
 9780847822102,
 9780890900956,
 9780878464340,
 9780913391198,
 9780886750732,
 9780967510651,
 9780937426333,
 9780300090574,
 9780824829292,
 9780834805392,
 9791163453192,
 9780735710078,
 9788960251991,
 9788960251748,
 9788962160604,
 9780764175763,
 9788975081750,
 9788975081729,
 9788975081736,
 9788975081767,
 9788962890273,
 9780385751162,
 9780899194264,
 9780688054359,
 9780394917498,
 9780763643126,
 9780060546403,
 9780439684316,
 9788962160468,
 9780060507855,
 9788995850329,
 9789191190943,
 9781797205670,
 9781946